H₀:	감염된 꿀벌 군집의 주변 밀원 구성은 우연적이다 (선호 밀원 등장 확률이 무작위와 동일)

H₁:	감염된 꿀벌 군집 주변에는 해당 종이 선호하는 밀원이 더 자주 등장한다

In [ ]:
import pandas as pd
from scipy.stats import binomtest

df_cerana = pd.read_csv("../../data/final/8_Apis_cerana_with_nearby_species.csv")
df_mellifera = pd.read_csv("../../data/final/8_Apis_mellifera_with_nearby_species.csv")

cerana_preferred = {'밤나무', '산밤나무', '약밤나무', '배추', '유채', '머귀나무',
                    '왕초피', '초피나무', '산초나무', '벼', '피마자'}
mellifera_preferred = {'족제비싸리', '아까지나무', '갈참나무', '굴참나무', '떡갈나무', '종가시나무',
                       '물참나무', '상수리나무', '신갈나무', '졸참나무', '가시나무', '붉가시나무',
                       '갈졸참나무', '떡갈참나무', '떡신갈나무', '주름잎', '누운주름잎', '덩굴장미',
                       '목향장미', '생열귀나무', '용가시나무', '인가목', '해당화', '고추', '애기똥풀'}
common_preferred = {'다래', '쥐다래', '개다래', '섬다래',
                    '감나무', '고욤나무', '개옺나무', '검양옺나무', '산검양옺나무',
                    '옺나무', '붉나무', '광대싸리'}
cerana_pref = cerana_preferred | common_preferred
mellifera_pref = mellifera_preferred | common_preferred

def extract_species_set(row):
    return {row[col] for col in row.index if col.startswith("nearby_") and pd.notna(row[col])}

df_cerana["species_set"] = df_cerana.apply(extract_species_set, axis=1)
df_mellifera["species_set"] = df_mellifera.apply(extract_species_set, axis=1)

df_cerana["has_preferred"] = df_cerana["species_set"].apply(lambda s: len(s & cerana_pref) > 0)
df_mellifera["has_preferred"] = df_mellifera["species_set"].apply(lambda s: len(s & mellifera_pref) > 0)

cer_success = df_cerana["has_preferred"].sum()
cer_total = len(df_cerana)
mel_success = df_mellifera["has_preferred"].sum()
mel_total = len(df_mellifera)
# 이항검정
test_cer = binomtest(cer_success, n=cer_total, p=0.5, alternative='greater')
test_mel = binomtest(mel_success, n=mel_total, p=0.5, alternative='greater')

print(f"[cerana] 선호 밀원 포함된 감염 군집 비율: {cer_success}/{cer_total} ({cer_success/cer_total:.2%}), p = {test_cer.pvalue:.4f}")
print(f"[mellifera] 선호 밀원 포함된 감염 군집 비율: {mel_success}/{mel_total} ({mel_success/mel_total:.2%}), p = {test_mel.pvalue:.4f}")


[cerana] 선호 밀원 포함된 감염 군집 비율: 652/1159 (56.26%), p = 0.0000
[mellifera] 선호 밀원 포함된 감염 군집 비율: 78/160 (48.75%), p = 0.6536
